In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from keras.optimizers import RMSprop, Adam
from utils.dataset import DataSet, LABELS
from models.textrnn import CuDNNGRULast as TextRNN
from models.sentiment_base import SCompositeModel as CompositeTextRNN
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

SAVED_PATH = os.path.join(SAVED_MODELS_PATH, 'textrnn', 'cudnngrulast_test')
if not os.path.exists(SAVED_PATH):
    os.makedirs(SAVED_PATH)

MODEL_PRE = 'cudnngrulast_'

LR = 1e-3
EPOCHS = 100
BATCH_SIZE = 64
FACTOR = 0.2

VECTOR_DIM = 300
EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem_%d.pkl' % VECTOR_DIM)
RESULT_CSV = os.path.join(RESULTS_PATH, 'cudnngrulast.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS

seq = 'words_seq'

train_x = np.array(list(train_with_seq[seq]))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq[seq]))
val_y = val_with_seq[y_cols]
test_x = np.array(list(test_with_seq[seq]))

comps = [(TextRNN, {"max_len":train_x.shape[1], 
                    'embedding': embedding,
                    'mask_zero': False})] * len(y_cols)
# comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = CompositeTextRNN(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq', saved_path = SAVED_PATH,
               model_pre = MODEL_PRE, lr = LR, epochs = EPOCHS, batch_size = BATCH_SIZE, optimizer = RMSprop,
               factor = FACTOR)

Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.5142 - acc: 0.8293 - val_loss: 0.2542 - val_acc: 0.9366


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Epoch 00001: _f1_monitor improved from -inf to 0.45910, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 2/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.2378 - acc: 0.9344 - val_loss: 0.2028 - val_acc: 0.9441

Epoch 00002: _f1_monitor improved from 0.45910 to 0.58156, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 3/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.2131 - acc: 0.9401 - val_loss: 0.1913 - val_acc: 0.9461

Epoch 00003: _f1_monitor improved from 0.58156 to 0.60998, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_0.h5
Epoch 4/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.2032 - acc: 0.9418 - val_loss: 0.1898 - val_acc: 0.9451

Epoch 00004: _f1_monitor


Epoch 00004: _f1_monitor improved from 0.61706 to 0.62507, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_2.h5
Epoch 5/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.2839 - acc: 0.9113 - val_loss: 0.2835 - val_acc: 0.9143

Epoch 00005: _f1_monitor improved from 0.62507 to 0.63374, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_2.h5
Epoch 6/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.2779 - acc: 0.9133 - val_loss: 0.2741 - val_acc: 0.9169

Epoch 00006: _f1_monitor improved from 0.63374 to 0.66758, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_2.h5
Epoch 7/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.2724 - acc: 0.9143 - val_loss: 0.2763 - val_acc: 0.9171

Epoch 00007: _f1_moni


Epoch 00002: _f1_monitor improved from 0.72659 to 0.75105, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_4.h5
Epoch 3/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4449 - acc: 0.8348 - val_loss: 0.4244 - val_acc: 0.8453

Epoch 00003: _f1_monitor improved from 0.75105 to 0.75444, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_4.h5
Epoch 4/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4317 - acc: 0.8402 - val_loss: 0.4139 - val_acc: 0.8455

Epoch 00004: _f1_monitor improved from 0.75444 to 0.75610, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_4.h5
Epoch 5/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4218 - acc: 0.8444 - val_loss: 0.4068 - val_acc: 0.8487

Epoch 00005: _f1_moni

...

105000/105000 [==============================] - 129s 1ms/step - loss: 0.4156 - acc: 0.8523 - val_loss: 0.4269 - val_acc: 0.8455

Epoch 00007: _f1_monitor improved from 0.66412 to 0.66427, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_10.h5
Epoch 8/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4111 - acc: 0.8551 - val_loss: 0.4156 - val_acc: 0.8441

Epoch 00008: _f1_monitor improved from 0.66427 to 0.66604, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_10.h5
Epoch 9/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4041 - acc: 0.8565 - val_loss: 0.4182 - val_acc: 0.8482

Epoch 00009: _f1_monitor improved from 0.66604 to 0.67456, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_10.h5
Epoch 10/100
105000/105000 [===

105000/105000 [==============================] - 129s 1ms/step - loss: 0.4749 - acc: 0.8262 - val_loss: 0.4963 - val_acc: 0.8232

Epoch 00009: _f1_monitor improved from 0.71634 to 0.71921, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_12.h5
Epoch 10/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.4709 - acc: 0.8281 - val_loss: 0.4915 - val_acc: 0.8227

Epoch 00010: _f1_monitor did not improve from 0.71921
Epoch 11/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4639 - acc: 0.8313 - val_loss: 0.4997 - val_acc: 0.8215

Epoch 00011: _f1_monitor did not improve from 0.71921
Epoch 12/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4579 - acc: 0.8342 - val_loss: 0.4985 - val_acc: 0.8153

Epoch 00012: _f1_monitor did not improve from 0.71921
Epoch 13/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4520 - a


Epoch 00007: _f1_monitor improved from 0.66874 to 0.67461, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_14.h5
Epoch 8/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.6094 - acc: 0.7798 - val_loss: 0.6181 - val_acc: 0.7778

Epoch 00008: _f1_monitor improved from 0.67461 to 0.67987, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_14.h5
Epoch 9/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.5987 - acc: 0.7842 - val_loss: 0.6369 - val_acc: 0.7797

Epoch 00009: _f1_monitor did not improve from 0.67987
Epoch 10/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.5872 - acc: 0.7887 - val_loss: 0.6309 - val_acc: 0.7767

Epoch 00010: _f1_monitor did not improve from 0.67987
Epoch 11/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.5792 - a

105000/105000 [==============================] - 130s 1ms/step - loss: 0.5162 - acc: 0.8165 - val_loss: 0.5570 - val_acc: 0.7995

Epoch 00009: _f1_monitor improved from 0.53694 to 0.54027, saving model to /root/mounted/projects/ai_challenger_sentiment/data/saved_models/textrnn/cudnngrulast_test/cudnngrulast_16.h5
Epoch 10/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.5073 - acc: 0.8195 - val_loss: 0.5633 - val_acc: 0.7985

Epoch 00010: _f1_monitor did not improve from 0.54027
Epoch 11/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.5003 - acc: 0.8240 - val_loss: 0.5704 - val_acc: 0.7962

Epoch 00011: _f1_monitor did not improve from 0.54027

Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 12/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.4817 - acc: 0.8288 - val_loss: 0.5810 - val_acc: 0.7925

Epoch 00012: _f1_monitor did not improve from 0.54027
Epoch 13/100

105000/105000 [==============================] - 129s 1ms/step - loss: 0.4509 - acc: 0.8358 - val_loss: 0.5193 - val_acc: 0.8059

Epoch 00009: _f1_monitor did not improve from 0.56300
Epoch 10/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.4435 - acc: 0.8405 - val_loss: 0.5227 - val_acc: 0.8073

Epoch 00010: _f1_monitor did not improve from 0.56300
Epoch 11/100
105000/105000 [==============================] - 129s 1ms/step - loss: 0.4397 - acc: 0.8425 - val_loss: 0.5259 - val_acc: 0.8055

Epoch 00011: _f1_monitor did not improve from 0.56300

Epoch 00011: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
The 18th grain: f1_score - 0.5629992247390625, acc - 0.8041333333333334
Train on 105000 samples, validate on 15000 samples
Epoch 1/100
105000/105000 [==============================] - 130s 1ms/step - loss: 0.5348 - acc: 0.8129 - val_loss: 0.4015 - val_acc: 0.8548

Epoch 00001: _f1_monitor improved from -inf to 0.62701, saving model to /ro

In [7]:
MODEL_PRE = os.path.join(SAVED_PATH, MODEL_PRE)
# comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
# comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)

In [9]:
val_preds = comp_model.predict(val_x)
test_preds = comp_model.predict(test_x)

In [10]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_y.values.T + 2, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))

The 0th f1: 0.6217684720371832
The 1th f1: 0.53109569566994
The 2th f1: 0.6781935447094292
The 3th f1: 0.6286916031329852
The 4th f1: 0.7709595355815511
The 5th f1: 0.7141099154332776
The 6th f1: 0.708214255289855
The 7th f1: 0.751510104740456
The 8th f1: 0.6860558724534589
The 9th f1: 0.6553736569443052
The 10th f1: 0.6767164497199212
The 11th f1: 0.7080554964919847
The 12th f1: 0.7192096358797667
The 13th f1: 0.6978057813463217
The 14th f1: 0.6798699669290538
The 15th f1: 0.6968601838613083
The 16th f1: 0.540272174918471
The 17th f1: 0.7205081377454897
The 18th f1: 0.5629992247390625
The 19th f1: 0.6758467905384353
The average f1 of val is 0.6712058249081128


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
res = test.copy()
for index, col in enumerate(LABELS):
    res[col] = test_preds[index] - 2

res.to_csv(RESULT_CSV, index = False)

In [12]:
print("Hello world")

Hello world
